#gureBERT

In [9]:
# gureBERT

!git clone --recursive  https://github.com/zmwebdev/gureBERT
%cd gureBERT

!pip install sentencepiece
!install -d spModels
# eu
!python src/sentence-split.py --config eu.config.ini --do_lower_case 
!python src/train-sentencepiece.py --config eu.config.ini

!head -n 100 spModels/eu.vocab

# en-eu
!python src/sentence-split.py --config en-eu.config.ini --do_lower_case 
!python src/train-sentencepiece.py --config en-eu.config.ini

!head -n 100 spModels/en-eu.vocab


<unk>	0
<s>	0
</s>	0
[PAD]	0
[CLS]	0
[SEP]	0
[MASK]	0
▁the	-2.94385
,	-3.12044
.	-3.36831
▁of	-3.63212
▁and	-3.88588
▁in	-3.94686
▁to	-4.14213
▁a	-4.18988
s	-4.33794
▁	-4.42405
▁is	-4.83168
▁"	-4.84885
-	-4.88602
▁as	-4.99159
▁(	-5.09098
▁was	-5.10418
▁for	-5.14361
ing	-5.15246
▁that	-5.17603
'	-5.23655
▁by	-5.25437
"	-5.30471
▁with	-5.32029
▁on	-5.42771
ed	-5.49214
▁are	-5.6122
▁from	-5.65004
▁*	-5.65699
▁it	-5.74278
▁eta	-5.80773
▁an	-5.81847
▁be	-5.82443
▁or	-5.82872
▁his	-5.8639
)	-5.90229
a	-5.94598
▁which	-5.96859
d	-5.97233
:	-5.98608
▁at	-6.00148
▁this	-6.02449
▁-	-6.0418
▁were	-6.06326
▁he	-6.0713
▁have	-6.13196
ly	-6.20551
▁has	-6.24478
▁not	-6.24927
n	-6.27705
▁also	-6.32563
▁had	-6.43683
▁but	-6.45083
▁one	-6.47153
;	-6.48428
▁been	-6.48897
e	-6.49357
▁its	-6.50068
ko	-6.51327
▁other	-6.5402
▁such	-6.5726
",	-6.6277
k	-6.63656
▁first	-6.65626
▁who	-6.68015
▁some	-6.68221
▁they	-6.69216
).	-6.70626
▁most	-6.7164
▁can	-6.73573
),	-6.75053
▁more	-6.75593
ir	-6.78541
ren	-6.804

In [18]:
import sys
sys.path.append("src")

import tokenization_sentencepiece as tokenization

tokenizer = tokenization.FullTokenizer(
    model_file="spModels/eu.model",
    vocab_file="spModels/eu.vocab",
    do_lower_case=True)

text1 = "Nere kotxea aitonaren etxe alboan dago, bere kolorea gorria da."

tokenizer.tokenize(text1)

Loaded a trained SentencePiece model.


['▁ne',
 're',
 '▁',
 'ko',
 'txea',
 '▁ai',
 'ton',
 'aren',
 '▁etxe',
 '▁alb',
 'o',
 'an',
 '▁dago',
 ',',
 '▁bere',
 '▁kolore',
 'a',
 '▁gorria',
 '▁da',
 '.']

In [15]:
tokenizer_en_eu = tokenization.FullTokenizer(
    model_file="spModels/en-eu.model",
    vocab_file="spModels/en-eu.vocab",
    do_lower_case=True)

text1 = "Nere kotxea aitonaren etxe alboan dago, bere kolorea gorria da."

tokenizer_en_eu.tokenize(text1)

Loaded a trained SentencePiece model.


['▁ne',
 're',
 '▁ko',
 'txea',
 '▁a',
 'it',
 'on',
 'aren',
 '▁etxe',
 '▁',
 'albo',
 'an',
 '▁dago',
 ',',
 '▁bere',
 '▁kolore',
 'a',
 '▁gorria',
 '▁da',
 '.']

In [23]:
text1 = "The Italian cities of Milan and Cortina d'Ampezzo are chosen as the joint hosts of the 2026 Winter Olympics and Winter Paralympics."

print("EN-EU: {}".format(len(tokenizer_en_eu.tokenize(text1))))
print("EU: {}".format(len(tokenizer.tokenize(text1))))

EN-EU: 31
EU: 44


In [24]:
from google.colab import auth
auth.authenticate_user()

W0629 17:11:49.993598 140062171948928 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [25]:
# Pre-Training
# https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/pretraining.ipynb

import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

#Check TPU devices

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.2.81.2:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5201560055092340784),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6671482654539601268),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15235922008977912019),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16587921834020422802),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 18249457006028921626),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 9769923856279949422),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 14707561106018069366),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 11546342222793515035),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 15225485262720

In [0]:
GS = 'gs://gurebert/gureBERT'
  
#!gsutil cp -r spModels $GS/
#!gsutil cp -r corpus $GS/

In [0]:
!python3 src/create_pretraining_data.py \
    --input_file=$GS/corpus/eu/2014wiki.eu.sent_splited \
    --output_file=$GS/pretraining.tf.data \
    --model_file=spModels/eu.model \
    --vocab_file=spModels/eu.vocab \
    --do_lower_case=True

W0629 14:23:14.996464 140356343957376 deprecation_wrapper.py:119] From src/create_pretraining_data.py:458: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 14:23:14.997114 140356343957376 deprecation_wrapper.py:119] From src/create_pretraining_data.py:424: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 14:23:14.997257 140356343957376 deprecation_wrapper.py:119] From src/create_pretraining_data.py:424: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

Loaded a trained SentencePiece model.
W0629 14:23:15.023280 140356343957376 deprecation_wrapper.py:119] From /content/gureBERT/src/tokenization_sentencepiece.py:115: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 14:23:15.085362 140356343957376 deprecation_wrapper.py:119] From src/create_pretraining_data.py:432: The name tf.gfile.Glob is deprecated. Please use tf.io.gfil

In [0]:
#!install -d gureBERT

!python src/run_pretraining.py \
  --config_file eu.congif.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=10000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
#  --num_train_steps=1000000 \


W0629 12:19:57.805771 139887738599296 deprecation_wrapper.py:119] From /content/gureBERT/src/../bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0629 12:19:57.808184 139887738599296 deprecation_wrapper.py:119] From src/run_pretraining.py:498: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 12:19:57.808862 139887738599296 deprecation_wrapper.py:119] From src/run_pretraining.py:413: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 12:19:57.809011 139887738599296 deprecation_wrapper.py:119] From src/run_pretraining.py:413: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0629 12:19:57.809142 139887738599296 deprecation_wrapper.py:119] From /content/gureBERT/src/../bert/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 12:19:57.809830 139

In [0]:
# pre training SQuAD

# https://github.com/google-research/bert#squad-20

#!git clone https://github.com/zmwebdev/bert.git
#%cd bert

!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
# evaluation script: download file from a url that returns a save dialog box : https://superuser.com/questions/795265/download-file-from-a-url-that-returns-a-save-dialog-box#795269
!wget --content-disposition https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/


--2019-06-29 14:24:12--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   175MB/s    in 0.2s    

2019-06-29 14:24:13 (175 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2019-06-29 14:24:14--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [0]:
# sentenpiece tokenizer  erabili behar da!!

!python bert/run_squad.py \
  --vocab_file=./spModels/eu.vocab \
  --bert_config_file=bert_config.json \
  --do_lower_case=True \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=0.1 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=gs://gurebert/gureBERT/squad/ \
  --use_tpu=True \
  --tpu_name=$TPU_NAME \
  --version_2_with_negative=True \
  #--init_checkpoint=$BERT_DIR \

W0629 14:24:22.660527 139909234767744 deprecation_wrapper.py:119] From /content/gureBERT/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0629 14:24:22.662883 139909234767744 deprecation_wrapper.py:119] From bert/run_squad.py:1283: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 14:24:22.663583 139909234767744 deprecation_wrapper.py:119] From bert/run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 14:24:22.663740 139909234767744 deprecation_wrapper.py:119] From bert/run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0629 14:24:22.663872 139909234767744 deprecation_wrapper.py:119] From /content/gureBERT/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 14:24:22.664730 139909234767744 deprecation_w

## wordpiece erabiliz

- https://towardsdatascience.com/pre-training-bert-from-scratch-with-cloud-tpu-6e2f71028379
- https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased-vocab.txt




In [0]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('./spModels/eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 14705
Sample tokens: ['emakumezko', '▁erla', 'olof', '▁40', 'ulara', '▁abiarazle', '▁uploadlogpage', '▁belle', '▁arazoen', '▁aipamena']


In [0]:
# hau errepasatu. adibidez @ ez dut ondo egiten... [UNK] agertzen da...
# adibidez, 

#import string
#def parse_sentencepiece_token(token):
#    if token.startswith("▁"):
#        return token[1:]
#    else:
#        if token in string.punctuation:
#            return token
#        else:
#            return "##" + token
          

def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token          

In [0]:
import string

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
# puntuazio sinboloak gehitu 
ctrl_symbols_end = list(string.punctuation)
#ctrl_symbols = ["[UNK]"]
bert_vocab = ctrl_symbols + bert_vocab + ctrl_symbols_end

#bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

14742


In [0]:
VOC_FNAME = "vocab.txt"
with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [0]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize("Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili ")

['ne',
 '##re',
 'ko',
 '##txea',
 'ait',
 '##ona',
 '##ren',
 'etxe',
 'alb',
 '##oan',
 'dago',
 '!',
 ',',
 'eta',
 'bere',
 'kolore',
 '##a',
 'gorria',
 'da',
 '.',
 'ni',
 '@',
 'ni',
 '.',
 'eus',
 'erabili']

In [0]:
!python bert/create_pretraining_data.py \
  --input_file=$GS/corpus/eu/2014wiki.eu.sent_splited \
  --output_file=$GS/wordpiece/pretraining.tf.data \
  --vocab_file=vocab.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=666 \
  --do_whole_word_mask=True \
  #--dupe_factor=5

W0629 14:27:51.331921 140698090002304 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:469: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 14:27:51.332679 140698090002304 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 14:27:51.332850 140698090002304 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0629 14:27:51.332988 140698090002304 deprecation_wrapper.py:119] From /content/gureBERT/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 14:27:51.387063 140698090002304 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0629 14:27:52.573521 14069809

In [0]:
'''
!python src/run_pretraining.py \
  --config_file eu.congif.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
'''

!python bert/run_pretraining.py \
  --input_file=$GS/wordpiece/pretraining.tf.data \ \
  --output_dir=$GS/wordpiece/model \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=10000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  #--num_train_steps=1000000 \
  #--init_checkpoint=$GS/wordpiece/model \

W0629 14:43:18.416723 139621729412992 deprecation_wrapper.py:119] From /content/gureBERT/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0629 14:43:18.417868 139621729412992 deprecation_wrapper.py:119] From bert/run_pretraining.py:493: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 14:43:18.418495 139621729412992 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 14:43:18.418647 139621729412992 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0629 14:43:18.418800 139621729412992 deprecation_wrapper.py:119] From /content/gureBERT/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 14:43:18.419515 13962172941299

### EN-EU

In [26]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('./spModels/en-eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 44999
Sample tokens: ['▁styling', 'lainn', '▁mayan', 'ned', 'spots', 'spectre', 'leninis', '▁diaspora', 'commenced', '▁subsidise']


In [0]:
# hau errepasatu. adibidez @ ez dut ondo egiten... [UNK] agertzen da...
# adibidez, 

#import string
#def parse_sentencepiece_token(token):
#    if token.startswith("▁"):
#        return token[1:]
#    else:
#        if token in string.punctuation:
#            return token
#        else:
#            return "##" + token
          

def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token          

In [41]:
import string

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
# puntuazio sinboloak gehitu 
ctrl_symbols_end = list(string.punctuation)
#ctrl_symbols = ["[UNK]"]
bert_vocab = ctrl_symbols + bert_vocab + ctrl_symbols_end

#bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

45036


In [0]:
VOC_FNAME = "vocab-en_eu.txt"
with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [43]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize("Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili ")

['ne',
 '##re',
 'ko',
 '##txea',
 'ai',
 '##ton',
 '##aren',
 'etxe',
 'alb',
 '##oa',
 '##n',
 'dago',
 '!',
 ',',
 'eta',
 'bere',
 'kolore',
 '##a',
 'gorria',
 'da',
 '.',
 'ni',
 '@',
 'ni',
 '.',
 'eu',
 '##s',
 'erabili']

In [48]:
FILES = "./corpus/en-eu/2014wiki.eu.sent_splited,./corpus/en-eu/2019wiki-10k.en.sent_splited"

!python bert/create_pretraining_data.py \
  --input_file={FILES} \
  --output_file=$GS/wordpiece/pretraining-en_eu.tf.data \
  --vocab_file=vocab-en_eu.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=666 \
  --do_whole_word_mask=True \
  #--dupe_factor=5

W0629 17:36:15.284230 140555365861248 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:469: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 17:36:15.285029 140555365861248 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 17:36:15.285196 140555365861248 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:437: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0629 17:36:15.285347 140555365861248 deprecation_wrapper.py:119] From /content/gureBERT/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 17:36:15.468693 140555365861248 deprecation_wrapper.py:119] From bert/create_pretraining_data.py:444: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0629 17:36:15.470420 14055536

In [0]:
'''
!python src/run_pretraining.py \
  --config_file eu.congif.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
'''

!python bert/run_pretraining.py \
  --input_file=$GS/wordpiece/pretraining-en_eu.tf.data \ \
  --output_dir=$GS/wordpiece/model-en_eu \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=10000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  #--num_train_steps=1000000 \
  #--init_checkpoint=$GS/wordpiece/model \

W0629 17:39:32.258262 139894899398528 deprecation_wrapper.py:119] From /content/gureBERT/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0629 17:39:32.259413 139894899398528 deprecation_wrapper.py:119] From bert/run_pretraining.py:493: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 17:39:32.260085 139894899398528 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 17:39:32.260240 139894899398528 deprecation_wrapper.py:119] From bert/run_pretraining.py:407: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0629 17:39:32.260382 139894899398528 deprecation_wrapper.py:119] From /content/gureBERT/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 17:39:32.261122 13989489939852

In [0]:
# data English da!! baino kodea badabil 

!python bert/run_squad.py \
  --vocab_file=vocab-en_eu.txt \
  --bert_config_file=bert_config.json \
  --do_lower_case=True \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=0.1 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=gs://gurebert/gureBERT/wordpiece/squad/ \
  --use_tpu=True \
  --tpu_name=$TPU_NAME \
  --version_2_with_negative=True \
  --init_checkpoint=$GS/wordpiece/model-en_eu \

W0629 14:39:54.130017 140691672389504 deprecation_wrapper.py:119] From /content/gureBERT/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0629 14:39:54.132371 140691672389504 deprecation_wrapper.py:119] From bert/run_squad.py:1283: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0629 14:39:54.133091 140691672389504 deprecation_wrapper.py:119] From bert/run_squad.py:1127: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0629 14:39:54.133235 140691672389504 deprecation_wrapper.py:119] From bert/run_squad.py:1127: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0629 14:39:54.133359 140691672389504 deprecation_wrapper.py:119] From /content/gureBERT/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0629 14:39:54.134238 140691672389504 deprecation_w

# gureBERT (japanese) erabiliz

In [0]:
#################################################################
# gureBERT japanese

!cd /content/

!git clone --recursive https://github.com/zmwebdev/bert-japanese
%cd bert-japanese



Cloning into 'bert-japanese'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 170 (delta 1), reused 6 (delta 1), pack-reused 161
Receiving objects: 100% (170/170), 261.38 KiB | 3.27 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Submodule 'bert' (https://github.com/google-research/bert.git) registered for path 'bert'
Cloning into '/content/gureBERT/bert-japanese/bert'...
remote: Enumerating objects: 329, done.        
remote: Total 329 (delta 0), reused 0 (delta 0), pack-reused 329        
Receiving objects: 100% (329/329), 239.27 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (188/188), done.
Submodule path 'bert': checked out 'f39e881b169b9d53bea03d2d341b31707a6c052b'
/content/gureBERT/bert-japanese


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!python3 src/data-download-and-extract.py
!bash src/file-preprocessing.sh

#!gsutil cp -r gs://gurebert/gureBERT/data/wiki/AA/wiki_00 data/wiki/AA/wiki_00

# Oharra: esaldiak lerro bakarrean jarri behar dira
#!cat data/wiki/AA/wiki_00

In [0]:
# wiki-eu.vocab eta wiki-eu.model sortzen ditu

!pip install sentencepiece
!python3 src/train-sentencepiece.py

In [0]:
!gsutil cp -r data/ gs://gurebert/gureBERT/
!gsutil cp -r model/ gs://gurebert/gureBERT/

In [0]:
!mkdir model
!gsutil cp gs://gurebert/gureBERT/model/wiki-eu.vocab model
!gsutil cp gs://gurebert/gureBERT/model/wiki-eu.model model
!head -n 50 model/wiki-eu.vocab

<unk>	0
<s>	0
</s>	0
[PAD]	0
[CLS]	0
[SEP]	0
[MASK]	0
,	-2.87014
.	-2.98912
▁eta	-3.5406
▁ziren	-4.15683
-	-4.26943
an	-4.4068
▁da	-4.40907
a	-4.47601
▁zen	-4.51427
ko	-4.52925
▁	-4.67755
▁zeuden	-4.76555
▁bat	-4.84966
▁"	-4.95145
"	-4.95743
▁1	-5.04005
ak	-5.2203
▁izan	-5.32102
▁zuen	-5.34175
en	-5.36722
ren	-5.38606
:	-5.53342
k	-5.56319
▁(	-5.58916
▁zituzten	-5.58963
▁zuten	-5.59859
▁etxek	-5.66741
)	-5.66971
▁du	-5.70751
▁zituen	-5.72891
▁enpresak	-5.73378
▁bere	-5.7694
▁2	-5.78944
▁dago	-5.79301
▁2009	-5.8295
▁ere	-5.84878
▁2007	-5.87422
n	-5.87535
▁pertsona	-5.88342
▁biztanle	-5.93646
aren	-5.97756
▁dira	-5.97904
▁ditu	-6.0186


In [0]:
#https://github.com/zmwebdev/bert-japanese/blob/master/notebook/check-trained-tokenizer.ipynb
!pip install sentencepiece

import sys
sys.path.append("src")

import tokenization_sentencepiece as tokenization

text1 = "Nere kotxea aitonaren etxe alboan dago"
text2 = "Gorria da gure etxearen inguruan dagoen lorearen kolorea"

tokenizer = tokenization.FullTokenizer(
    model_file="model/wiki-eu.model",
    vocab_file="model/wiki-eu.vocab",
    do_lower_case=True)

tokenizer.tokenize(text1)

Loaded a trained SentencePiece model.


['▁nere', '▁kotxe', 'a', '▁aitona', 'ren', '▁etxe', '▁alboan', '▁dago']

In [0]:
tokenizer.tokenize(text2)

['▁gorria',
 '▁da',
 '▁gure',
 '▁etxearen',
 '▁inguruan',
 '▁dagoen',
 '▁lore',
 'aren',
 '▁kolorea']

In [0]:
#do_lower_case=False jarrita tokenizazioa okerragoa da
tokenizer = tokenization.FullTokenizer(
    model_file="model/wiki-eu.model",
    vocab_file="model/wiki-eu.vocab",
    do_lower_case=False)

tokenizer.tokenize(text1)

['▁', 'N', 'ere', '▁kotxe', 'a', '▁aitona', 'ren', '▁etxe', '▁alboan', '▁dago']

In [0]:
tokenizer.tokenize(text2)

['▁',
 'G',
 'orri',
 'a',
 '▁da',
 '▁gure',
 '▁etxearen',
 '▁inguruan',
 '▁dagoen',
 '▁lore',
 'aren',
 '▁kolorea']

In [0]:
# convert to WordPiece (kodean errorea dago). begiratu https://github.com/kwonmha/bert-vocab-builder/pull/4#issue-291306156

!git clone https://github.com/kwonmha/bert-vocab-builder.git

!python bert-vocab-builder/subword_builder.py \
--corpus_filepattern model/wiki-eu.vocab \
--output_filename model/wiki-eu-wordpiece.vocab
#--min_count {minimum_subtoken_counts}

# https://github.com/kwonmha/bert-vocab-builder/pull/4#issue-291306156

fatal: destination path 'bert-vocab-builder' already exists and is not an empty directory.
Traceback (most recent call last):
  File "bert-vocab-builder/subword_builder.py", line 33, in <module>
    import text_encoder
  File "/content/bert-japanese/bert-vocab-builder/text_encoder.py", line 647
    <<<<<<< HEAD
     ^
SyntaxError: invalid syntax


In [0]:
#Creating data for pretraining
#Create .tfrecord files for pretraining. For longer sentence data, replace the value of max_seq_length with 512.
!cat creating_data_for_pretraining.sh
!bash creating_data_for_pretraining.sh

for DIR in $( find data/wiki/ -mindepth 1 -type d ); do 
  python3 src/create_pretraining_data.py \
    --input_file=${DIR}/all.txt \
    --output_file=${DIR}/all-maxseq128.tfrecord \
    --model_file=./model/wiki-eu.model \
    --vocab_file=./model/wiki-eu.vocab \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=12345 \
    --dupe_factor=5
done

In [0]:
# copy to gs
!gsutil cp -r data gs://gurebert/gureBERT
!gsutil cp -r model gs://gurebert/gureBERT
  

In [0]:
# Pre-Training
# https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/pretraining.ipynb

import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

#Check TPU devices

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  


TPU address is grpc://10.16.115.170:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5460104680638402674),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16051411686286296001),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1933247979704415977),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11046566402910946519),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 13421296677076221120),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5352660506872180047),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 7874427115731877787),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15986719012258325089),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 76127872665

In [0]:
INPUT_DATA_GCS = 'gs://gurebert/gureBERT/data/wiki'
TARGET_DIRS = [
  'AA',
  'AB',
  'AC'
]

MAX_SEQ_LEN = 128
#MAX_SEQ_LEN = 512


INPUT_FILE = ','.join( [ '{}/{}/all-maxseq{}.tfrecord'.format(INPUT_DATA_GCS, elem, MAX_SEQ_LEN) for elem in TARGET_DIRS] )

OUTPUT_GCS = 'gs://gurebert/gureBERT/model'

In [0]:
# Adding whole word masking aldaketa egin dute BERT-en. update egin behar da!!!
# Begiratu https://github.com/google-research/bert/commit/0fce551b55caabcfba52c61e18f34b541aef186a

!python src/run_pretraining.py \
  --input_file={INPUT_FILE} \
  --output_dir={OUTPUT_GCS} \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --num_tpu_cores=8 \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length={MAX_SEQ_LEN} \
  --max_predictions_per_seq=20 \
  --num_train_steps=1400000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4

W0625 19:38:15.208214 140282338469760 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0625 19:38:15.209448 140282338469760 deprecation_wrapper.py:119] From src/run_pretraining.py:497: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0625 19:38:15.210220 140282338469760 deprecation_wrapper.py:119] From src/run_pretraining.py:412: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0625 19:38:15.210398 140282338469760 deprecation_wrapper.py:119] From src/run_pretraining.py:412: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0625 19:38:15.210564 140282338469760 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0625 19:38:15.

In [0]:
##########################################
##########################################

# BERT-ekin konpatiblea den vocab sortzeko prozedura. wordpiece _ -> ##. 
# Honen abantaila BERT kodearen aldaketak eta run_squad.py, ... ezer aldatu gabe ibiliko dela da.
# berez ez da wordpiece, sentencepiece baizik baina sintaktikoki konpatiblea
# ikusi: https://colab.research.google.com/drive/1-uLyGTnz2K4gx3su6Qc00in9QCa4b5Kh#scrollTo=9S4CiOh3RzFW


In [0]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('model/wiki-eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 31999
Sample tokens: ['.036', '▁ukuilu', 'lok', '▁ahaltsua', '▁batzuei', '▁erabili', '▁chapman', '▁tv', '▁pe', '▁polemika']


In [0]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token

In [0]:
bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

32004


In [0]:
VOC_FNAME = "vocab.txt" #@param {type:"string"}

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [0]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize("Nere kotxea aitonaren etxe alboan dago")



['nere', 'kotxe', '##a', 'aitona', '##ren', 'etxe', 'alboan', 'dago']

In [0]:
#!gsutil cp -r gs://gurebert/gureBERT/data/wiki/AA/wiki_00 wiki_00
# ez du wiki-eu.model behar, vocab.txt-ekin nahikoa da

!python bert/create_pretraining_data.py \
  --input_file=wiki_00 \
  --output_file=/tmp/tf_examples.tfrecord \
  --vocab_file=vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5

#############################################################
#############################################################

tf_examples.tfrecord  tmpcqwLhN
